In [1]:
import torch
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import sctokenizer
import spacy
import string
import re
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.init as weight_init
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import optim

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 Ti'

In [3]:
df = pd.read_csv("E:/BITS/2nd Year/Industry project/Dataset/c_data_twocols.zip")

In [4]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [5]:
df.head()

,code,comment
0,struct _CameraFile {\n char\t\tmime_typ...,NaN
1,struct _CameraFilesystem {\n\tCameraFilesystem...,The internals of the #CameraFilesystem are onl...
2,struct _CameraWidget {\n\tCameraWidgetType typ...,CameraWidget: The internals of the #CameraWidg...
3,struct _entry {\n\tchar *name;\n\tchar *value;...,NaN
4,\tchar error[2048];\t/**< Internal kept error ...,Internal kept error message.


In [6]:
df = df.dropna(axis=0)

In [7]:
df['code'] = df['code'].astype('str')
df['comment'] = df['comment'].astype('str')

In [8]:
df_train = df.head(10000)

In [9]:
from sctokenizer import CppTokenizer

ctokenizer = CppTokenizer()

In [10]:
from spacy.lang.en.stop_words import STOP_WORDS

spacy_nlp = spacy.load('en_core_web_sm')

punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [11]:
def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [12]:
def str_code(x):
    tokens = ctokenizer.tokenize(x)
    string_tok = [str(i.token_value)+" "+str(i.token_type) for i in tokens]
    return string_tok

In [13]:
str_code(df_train.iat[0,0])

['struct TokenType.KEYWORD',
 '_ TokenType.SPECIAL_SYMBOL',
 'CameraFilesystem TokenType.IDENTIFIER',
 '{ TokenType.SPECIAL_SYMBOL',
 'CameraFilesystemFolder TokenType.IDENTIFIER',
 '* TokenType.OPERATOR',
 'rootfolder TokenType.IDENTIFIER',
 '; TokenType.SPECIAL_SYMBOL',
 'CameraFilesystemFile TokenType.IDENTIFIER',
 '* TokenType.OPERATOR',
 'lru_first TokenType.IDENTIFIER',
 '; TokenType.SPECIAL_SYMBOL',
 'CameraFilesystemFile TokenType.IDENTIFIER',
 '* TokenType.OPERATOR',
 'lru_last TokenType.IDENTIFIER',
 '; TokenType.SPECIAL_SYMBOL',
 'unsigned TokenType.KEYWORD',
 'long TokenType.KEYWORD',
 'int TokenType.KEYWORD',
 'lru_size TokenType.IDENTIFIER',
 '; TokenType.SPECIAL_SYMBOL',
 'CameraFilesystemGetInfoFunc TokenType.IDENTIFIER',
 'get_info_func TokenType.IDENTIFIER',
 '; TokenType.SPECIAL_SYMBOL',
 'CameraFilesystemSetInfoFunc TokenType.IDENTIFIER',
 'set_info_func TokenType.IDENTIFIER',
 '; TokenType.SPECIAL_SYMBOL',
 'CameraFilesystemListFunc TokenType.IDENTIFIER',
 'file_li

In [14]:
df_train['comment_token'] = df_train['comment'].map(lambda x: spacy_tokenizer(x))
df_train['code_token'] = df_train['code'].map(lambda x: str_code(x))

In [15]:
df_train.head()

,code,comment,comment_token,code_token
1,struct _CameraFilesystem {\n\tCameraFilesystem...,The internals of the #CameraFilesystem are onl...,"[internal, camerafilesystem, visible, access, ...","[struct TokenType.KEYWORD, _ TokenType.SPECIAL..."
2,struct _CameraWidget {\n\tCameraWidgetType typ...,CameraWidget: The internals of the #CameraWidg...,"[camerawidget, internal, camerawidget, visible...","[struct TokenType.KEYWORD, _ TokenType.SPECIAL..."
4,\tchar error[2048];\t/**< Internal kept error ...,Internal kept error message.,"[internal, error, message]","[char TokenType.KEYWORD, error TokenType.KEYWO..."
5,\tstruct _GPPortInfo info;\t/**< Internal port...,Internal port information of this port.,"[internal, port, information, port]","[struct TokenType.KEYWORD, _ TokenType.SPECIAL..."
6,\tGPPortOperations *ops;\t/**< Internal port o...,Internal port operations.,"[internal, port, operation]","[GPPortOperations TokenType.IDENTIFIER, * Toke..."


In [16]:
counts_com = Counter()
for index, row in df_train.iterrows():
    counts_com.update(row['comment_token'])

In [17]:
print("num_words before:",len(counts_com.keys()))
for word in list(counts_com):
    if counts_com[word] < 2:
        del counts_com[word]
print("num_words after:",len(counts_com.keys()))

num_words before: 6881
num_words after: 3435


In [18]:
counts_code = Counter()
for index, row in df_train.iterrows():
    counts_code.update(row['code_token'])

In [19]:
print("num_words before:",len(counts_code.keys()))
for word in list(counts_code):
    if counts_code[word] < 2:
        del counts_code[word]
print("num_words after:",len(counts_code.keys()))

num_words before: 44062
num_words after: 25180


In [20]:
vocab2index_com = {"":0, "UNK":1}
words_com = ["", "UNK"]
for word in counts_com:
    vocab2index_com[word] = len(words_com)
    words_com.append(word)

In [21]:
vocab2index_code = {"":0, "UNK":1}
words_code = ["", "UNK"]
for word in counts_code:
    vocab2index_code[word] = len(words_code)
    words_code.append(word)

In [22]:
l_com = []
for index, row in df_train.iterrows():
    l_com.append(len(row['comment_token']))
print(f"Mean = {np.mean(l_com)}")
print(f"Median = {np.median(l_com)}")
print(f"Max = {np.max(l_com)}")

Mean = 10.3102
Median = 6.0
Max = 375


In [23]:
l_code = []
for index, row in df_train.iterrows():
    l_code.append(len(row['code_token']))
print(f"Mean = {np.mean(l_code)}")
print(f"Median = {np.median(l_code)}")
print(f"Max = {np.max(l_code)}")

Mean = 156.7234
Median = 80.0
Max = 5554


In [24]:
def encode_sentence(tokenized, vocab2index, N=250):
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [25]:
def encode_code(tokenized, vocab2index, N=3000):
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [26]:
df_train['comment_vec'] = df_train['comment_token'].apply(lambda x: np.array(encode_sentence(x,vocab2index_com )))
df_train['code_vec'] = df_train['code_token'].apply(lambda x: np.array(encode_code(x,vocab2index_code )))

<ipython-input-26-de7353da255d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df_train['comment_vec'] = df_train['comment_token'].apply(lambda x: np.array(encode_sentence(x,vocab2index_com )))
<ipython-input-26-de7353da255d>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df_train['code_vec'] = df_train['code_token'].apply(lambda x: np.array(encode_code(x,vocab2index_code )))


In [27]:
df_train.head()

,code,comment,comment_token,code_token,comment_vec,code_vec
1,struct _CameraFilesystem {\n\tCameraFilesystem...,The internals of the #CameraFilesystem are onl...,"[internal, camerafilesystem, visible, access, ...","[struct TokenType.KEYWORD, _ TokenType.SPECIAL...","[[2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 3, 4, 5, 1, 6, 7, 8, 9, 6, 1, 8, 9, 6, 1,..."
2,struct _CameraWidget {\n\tCameraWidgetType typ...,CameraWidget: The internals of the #CameraWidg...,"[camerawidget, internal, camerawidget, visible...","[struct TokenType.KEYWORD, _ TokenType.SPECIAL...","[[9, 2, 9, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0,...","[[2, 3, 29, 5, 30, 31, 8, 32, 33, 34, 35, 36, ..."
4,\tchar error[2048];\t/**< Internal kept error ...,Internal kept error message.,"[internal, error, message]","[char TokenType.KEYWORD, error TokenType.KEYWO...","[[2, 10, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[32, 58, 34, 59, 36, 8, 41, 42, 0, 0, 0, 0, 0..."
5,\tstruct _GPPortInfo info;\t/**< Internal port...,Internal port information of this port.,"[internal, port, information, port]","[struct TokenType.KEYWORD, _ TokenType.SPECIAL...","[[2, 12, 13, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[2, 3, 60, 37, 8, 41, 42, 0, 0, 0, 0, 0, 0, 0..."
6,\tGPPortOperations *ops;\t/**< Internal port o...,Internal port operations.,"[internal, port, operation]","[GPPortOperations TokenType.IDENTIFIER, * Toke...","[[2, 12, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[61, 6, 62, 8, 41, 42, 0, 0, 0, 0, 0, 0, 0, 0..."


In [28]:
code = []
comment = []
for index,row in df_train.iterrows():
    code.append((row['code_vec'])[0])
    comment.append((row['comment_vec'])[0])

In [35]:
code_len = code[0].shape
comment_len = comment[0].shape

In [86]:
class SeqEncoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, n_layers=1):
        super(SeqEncoder, self).__init__()
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True, bidirectional=True)
        self.init_weights()
        
    def init_weights(self):
        nn.init.uniform_(self.embedding.weight, -0.1, 0.1)
        nn.init.constant_(self.embedding.weight[0], 0)
        for name, param in self.lstm.named_parameters():
            if 'weight' in name or 'bias' in name: 
                param.data.uniform_(-0.1, 0.1)

    def forward(self, inputs, input_lens=None): 
        '''
        input_lens: [batch_size]
        '''
        batch_size, seq_len=inputs.size()
        inputs = self.embedding(inputs)  # input: [batch_sz x seq_len]  embedded: [batch_sz x seq_len x emb_sz]
        inputs = F.dropout(inputs, 0.25, self.training)
        
        #if input_lens is not None:# sort and pack sequence 
        #    input_lens_sorted, indices = input_lens.sort(descending=True)
        #    inputs_sorted = inputs.index_select(0, indices)        
        #    inputs = pack_padded_sequence(inputs_sorted, input_lens_sorted.data.tolist(), batch_first=True)
            
        hids, (h_n, c_n) = self.lstm(inputs)  
        
        #if input_lens is not None: # reorder and pad
        #    _, inv_indices = indices.sort()
        #    hids, lens = pad_packed_sequence(hids, batch_first=True) # hids:[batch_size x seq_len x (n_dir*hid_sz)](biRNN)
        #    hids = F.dropout(hids, p=0.25, training=self.training)
        #    hids = hids.index_select(0, inv_indices)
        #    h_n = h_n.index_select(1, inv_indices)
        h_n = h_n.view(self.n_layers, 2, batch_size, self.hidden_size) #[n_layers x n_dirs x batch_sz x hid_sz]
        h_n = h_n[-1] 
        encoding3 = torch.cat((h_n[0], h_n[1]), dim=1)
        return encoding3 

In [87]:
class BOWEncoder(nn.Module):
  
    def __init__(self, vocab_size, emb_size, hidden_size):
        super(BOWEncoder, self).__init__()
        self.emb_size=emb_size
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size)
        
        self.init_weights()
        
    def init_weights(self):
        nn.init.uniform_(self.embedding.weight, -0.1, 0.1)
        nn.init.constant_(self.embedding.weight[0], 0)
        
    def forward(self, input, input_len=None): 
        batch_size, seq_len =input.size()
        embedded = self.embedding(input)  
        embedded= F.dropout(embedded, 0.25, self.training) 
        maxpooling = nn.MaxPool1d(kernel_size = seq_len, stride=seq_len)
        output_pool = maxpooling(embedded.transpose(1,2)).squeeze(2) 
        encoding = output_pool
        return encoding

In [101]:
class JointEmbedder(nn.Module):

    def __init__(self):
        super(JointEmbedder, self).__init__()
        self.margin = 0.04
               
        
        self.code_encoder=BOWEncoder(10000,512,512)
        self.comment_encoder=SeqEncoder(10000,512,1024)

        
        self.w_code = nn.Linear(512, 512)
        self.w_comment = nn.Linear(2*1024, 512)
        
        self.init_weights()
        
    def init_weights(self):# Initialize Linear Weight 
        for m in [self.w_code]:        
            m.weight.data.uniform_(-0.1, 0.1)#nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0.) 
            
    def code_encoding(self, code_tokens, code_len):
        print(code_tokens.shape)
        code_repr=self.code_encoder(code_tokens, code_len)
        print(code_repr)
        code_repr = self.w_code(code_repr)
        return code_repr
        
    def comment_encoding(self, comment, comment_len):
        comment_repr=self.comment_encoder(comment, comment_len)
        comment_repr=self.w_comment(comment_repr)
        return comment_repr
    
    def similarity(self, code_vec, comment_vec):
   
            return F.cosine_similarity(code_vec, comment_vec)
      
    
    def forward(self, code, code_len, comment_anchor, comment_anchor_len, comment_neg, comment_neg_len):
        batch_size=code.size(0)
        code_repr=self.code_encoding(code, code_len)
        comment_anchor_repr=self.comment_encoding(comment_anchor, comment_anchor_len)
        comment_neg_repr=self.comment_encoding(comment_neg, comment_neg_len)
    
        anchor_sim = self.similarity(code_repr, comment_anchor_repr)
        neg_sim = self.similarity(code_repr, comment_neg_repr) # [batch_sz x 1]
        
        loss=(self.margin-anchor_sim+neg_sim).clamp(min=1e-6).mean()
        
        return loss,anchor_sim,neg_sim

In [102]:
model = JointEmbedder()

In [103]:
testcode = torch.from_numpy(np.array(code[0:2]))
testcomm = torch.from_numpy(np.array(comment[0:2]))
testnegcomm = torch.from_numpy(np.array(comment[4:6]))

In [104]:
model(testcode,code_len,testcomm,comment_len,testnegcomm,comment_len)

torch.Size([2, 3000])
tensor([[0.1283, 0.1294, 0.1229,  ..., 0.1331, 0.1236, 0.1143],
        [0.1307, 0.1322, 0.1289,  ..., 0.1302, 0.1096, 0.1148]],
       grad_fn=<SqueezeBackward1>)


(tensor(0.0198, grad_fn=<MeanBackward0>),
 tensor([0.0610, 0.0550], grad_fn=<DivBackward0>),
 tensor([0.0412, 0.0343], grad_fn=<DivBackward0>))